## **Comportamiento Radiación Solar INMET**

In [ ]:
fig_INMET = px.line(
    df_INMET,
    x='datetime',
    y='RADIACAO_GLOBAL',
    title='Radiación Global (KJ/m²) por Hora INMET (2019 a Febrero 2025)',
    labels={
        'datetime': 'Fecha y Hora',
        'RADIACAO_GLOBAL': 'Radiación (KJ/m²)'
    }
)
fig_INMET.update_layout(template='plotly_white', width=1100, height=500)
fig_INMET.show()

## **Comportamiento Radiación Solar NASA**

In [ ]:
fig_nasa = px.line(
    df_NASA,
    x='datetime',
    y='ALLSKY_SFC_SW_DWN',
    title='Radiación Horaria NASA (2019-2025)',
    labels={'datetime': 'Fecha y hora', 'ALLSKY_SFC_SW_DWN': 'Radiación (W/m²)'}
)
fig_nasa.update_layout(template='plotly_white', width=1100, height=500)
fig_nasa.show()

## **Comportamiento de Radiacioón Solar INMET vs NASA**

In [ ]:
df_INMET['datetime'] = pd.to_datetime(df_INMET['datetime'].dt.floor('H'))
df_NASA['datetime'] = pd.to_datetime(df_NASA['datetime'].dt.floor('H'))

df_INMET['RADIACAO GLOBAL (KJ/m²)'] = df_INMET['RADIACAO_GLOBAL'].fillna(0)

df_comparado = pd.merge(
    df_INMET[['datetime', 'RADIACAO_GLOBAL']],
    df_NASA[['datetime', 'ALLSKY_SFC_SW_DWN']],
    on='datetime',
    how='inner'
)
print(df_comparado.shape)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['RADIACAO_GLOBAL'],
    mode='lines',
    name='INMET',
    line=dict(color='blue'),
    opacity=0.7
))
fig.add_trace(go.Scatter(
    x=df_comparado['datetime'],
    y=df_comparado['ALLSKY_SFC_SW_DWN'],
    mode='lines',
    name='NASA',
    line=dict(color='orange'),
    opacity=0.7
))
fig.update_layout(
    title='Evolución de Radiación Horaria - INMET vs. NASA (2019-2025)',
    xaxis_title='Fecha y Hora',
    yaxis_title='Radiación (KJ/m²)',
    legend_title='Fuente',
    template='plotly_white',
    height=500,
    width=1100
)
fig.show()

## **Distribución de Radiación Solar INMET vs NASA**

Las distribuciones de radiación solar provenientes de las fuentes **INMET** (`RADIACAO_GLOBAL`) y **NASA** (`ALLSKY_SFC_SW_DWN`) presentan comportamientos claramente *no normales*, como se observa en los histogramas. Ambas variables muestran una **alta frecuencia de valores cercanos a cero**, posiblemente asociada a registros nocturnos o condiciones de nubosidad extrema, seguidos de distribuciones **asimétricas y multimodales**.

En particular, la distribución de INMET evidencia una dispersión más amplia con mayor presencia de valores altos (hasta **4000 KJ/m²**), mientras que la distribución de NASA, medida en **W/m²**, refleja una estructura más suave y densa en rangos medios, aunque también con múltiples picos.

Estos patrones justifican el uso de **pruebas no paramétricas** y técnicas de modelado que *no asuman normalidad* en los datos, como la prueba de **Kolmogórov-Smirnov** o modelos basados en aprendizaje automático flexible.


In [ ]:
df_comparado['RADIACAO_GLOBAL'] = df_comparado['RADIACAO_GLOBAL'].fillna(0)
rad1 = df_comparado['RADIACAO_GLOBAL']
rad1 = rad1[rad1 > 0]

rad2 = df_comparado['ALLSKY_SFC_SW_DWN']
rad2 = rad2[rad2 > 0]
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

sns.histplot(rad1, bins=60, kde=True, ax=axes[0], color='blue')
axes[0].set_title('RADIACAO_GLOBAL (filtrado > 0)')
axes[0].set_xlabel('Radiación (KJ/m²)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True)

sns.histplot(rad2, bins=60, kde=True, ax=axes[1], color='orange')
axes[1].set_title('ALLSKY_SFC_SW_DWN (filtrado > 0)')
axes[1].set_xlabel('Radiación (W/m²)')
axes[1].set_ylabel('Frecuencia')
axes[1].grid(True)
plt.suptitle("Distribuciones de Radiación Solar ", fontsize=20)
plt.tight_layout()
plt.show()

## **Pruebas de Normalidad de Radiacioón Solar INMET vs NASA**

La prueba de **Jarque-Bera** es un método estadístico utilizado para evaluar la **normalidad de una distribución de datos** a partir de sus momentos estadísticos, específicamente la **asimetría** (*skewness*) y la **curtosis**. La lógica de esta prueba se basa en comparar los valores empíricos de la asimetría y la curtosis con los valores esperados bajo una distribución normal (es decir, \( S = 0 \) y \( K = 3 \)). Si las desviaciones son grandes, el estadístico de Jarque-Bera será elevado y su correspondiente valor \( p \) indicará **rechazo de la hipótesis nula**, sugiriendo que los datos **no provienen de una distribución normal**.


### **Fórmula de la prueba de Jarque-Bera**

La prueba de **Jarque-Bera** permite evaluar si una muestra sigue una distribución normal, basándose en los valores de asimetría (skewness) y curtosis (kurtosis). Su fórmula es la siguiente:

$$
JB = \frac{n}{6} \left( S^2 + \frac{(K - 3)^2}{4} \right)
$$

donde:

- \( JB \) es el estadístico de la prueba de Jarque-Bera  
- \( n \) es el tamaño de la muestra  
- \( S \) es el coeficiente de **asimetría** (skewness)  
- \( K \) es el coeficiente de **curtosis**  
- Para una distribución normal, se espera que \( S = 0 \) y \( K = 3 \)

**Criterio de decisión:**

- \( H_0 \): los datos provienen de una distribución normal  
- Si el valor p es menor que 0.05 → **se rechaza \( H_0 \)** (los datos **no son normales**)  
- Si el valor p es mayor que 0.05 → **no se rechaza \( H_0 \)** (los datos **pueden considerarse normales**)


In [ ]:
stat_inmet, p_inmet = jarque_bera(df_comparado['RADIACAO_GLOBAL'])
stat_nasa, p_nasa = jarque_bera(df_comparado['ALLSKY_SFC_SW_DWN'])
print(f" Jarque-Bera - INMET")
print(f"  Estadístico: {stat_inmet:.4f} | p-valor: {p_inmet:.4e} → {'Distribución normal' if p_inmet > 0.05 else 'NO normal'}")
print(f"\n Jarque-Bera - NASA")
print(f"  Estadístico: {stat_nasa:.4f} | p-valor: {p_nasa:.4e} → {'Distribución normal' if p_nasa > 0.05 else 'NO normal'}")


### **Prueba de Kolmogórov-Smirnov para Dos Muestras**

Luego de aplicar la prueba de **Jarque-Bera** y confirmar que las distribuciones de radiación solar (INMET y NASA) **no siguen una distribución normal**, se hace necesario emplear métodos estadísticos **no paramétricos** para comparar ambas fuentes de datos.

En este contexto, la **prueba de Kolmogórov–Smirnov (KS)** de dos muestras resulta relevante, ya que permite evaluar si ambas distribuciones **provienen de la misma población**, sin asumir normalidad ni homogeneidad de varianzas. Esta prueba compara las funciones de distribución empírica (ECDF) de cada conjunto de datos y mide la **mayor diferencia absoluta entre ellas**, lo que la convierte en una herramienta robusta para detectar discrepancias en forma, posición o dispersión entre las distribuciones.

Aplicar esta prueba permite establecer con mayor objetividad si las diferencias observadas entre las mediciones de radiación solar de INMET y NASA son **estadísticamente significativas** desde una perspectiva no paramétrica.


La **prueba de Kolmogórov-Smirnov (KS)** de dos muestras es una prueba **no paramétrica** utilizada para comparar si **dos distribuciones de datos continuos** provienen de la misma población.

**Hipótesis:**

- $H_0$: Ambas muestras provienen de la misma distribución.
- $H_1$: Las muestras provienen de distribuciones diferentes.

**Estadístico de Prueba:**

La fórmula del estadístico de KS es:

$$
D = \sup_x |F_1(x) - F_2(x)|
$$

Donde:

- $F_1(x)$ y $F_2(x)$ son las funciones de distribución empíricas (ECDF) de cada muestra.
- $\sup$ representa el máximo valor de la diferencia absoluta entre las dos ECDFs.

In [ ]:
ks_stat, p_value = ks_2samp(
   df_comparado['RADIACAO_GLOBAL'],
    df_comparado['ALLSKY_SFC_SW_DWN']
)
print(f"KS Statistic: {ks_stat:.4f}")
print(f"Valor p: {p_value:.4f}")

if p_value < 0.05:
    print("Las distribuciones de radiación entre INMET y NASA son estadísticamente diferentes.")
else:
    print("No se detectan diferencias significativas entre las distribuciones de radiación.")
